In [48]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# Define your MoireCNN model class as provided
class MoireCNN(nn.Module):
    def __init__(self):
        super(MoireCNN, self).__init__()
        
        # Encoder part (downsampling)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        
        # Decoder part (upsampling)
        self.deconv1 = nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1)  # Adjusted kernel size
        self.deconv2 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1)  # Adjusted kernel size
        self.deconv3 = nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1)    # Adjusted kernel size

        # ReLU activation
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        
        # Upsampling layer to scale up the final output to 256x256
        self.upsample = nn.Upsample(size=(256, 256), mode='bilinear', align_corners=False)
    
    def forward(self, x):
        # Encoder path
        x = self.relu(self.conv1(x))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        
        # Decoder path (upsample)
        x = self.relu(self.deconv1(x))
        x = self.relu(self.deconv2(x))
        x = self.deconv3(x)  # Final output layer to match 3 channels
        
        # Upsample to 256x256
        x = self.upsample(x)
        
        return x

# Load the model weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the model and load saved weights
model = MoireCNN().to(device)
model.load_state_dict(torch.load('moire_model.pth', map_location=device))
model.eval()  # Set the model to evaluation mode


MoireCNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (deconv1): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (deconv2): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (deconv3): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (upsample): Upsample(size=(256, 256), mode='bilinear')
)

In [49]:
# Define the preprocessing transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize to 256x256 since your model uses this size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalization
])

# Function to predict if an image contains moiré or not
def predict_image(image_path):
    image = Image.open(image_path).convert('RGB')  # Ensure the image is in RGB mode
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension and move to device
    with torch.no_grad():
        output = model(image)  # Run the image through the model
    return output


In [50]:
import torch.nn.functional as F

# Function to classify whether moiré is present or not
def classify_moire(output):
    # You can use a threshold here to classify. Let's assume:
    # If the max value in the output is greater than a certain threshold, we classify it as moiré.
    threshold = 1.5  # Set a threshold, adjust as needed
    output_max = torch.max(output).item()  # Get the maximum value of the output

    if output_max > threshold:
        return 'Moiré'
    else:
        return 'No Moiré'

def visualize_prediction(image_path):
    image = Image.open(image_path).convert('RGB')
    plt.imshow(image)
    plt.axis('off')
    plt.show()

    # Get the output of the model
    output = predict_image(image_path)
    
    # Classify the output
    prediction = classify_moire(output)
    print(f"Prediction: {prediction}")

from pathlib import Path

def classify_images_in_folder(folder_path):
    folder = Path(folder_path)
    for img_path in folder.glob('*.jpg'):  # Change to *.png or any format you have
        print(f"Classifying {img_path.name}...")
        output = predict_image(img_path)
        prediction = classify_moire(output)
        print(f"Image {img_path.name} is classified as: {prediction}")

# Example usage: Replace 'your_images_folder' with the folder containing images
classify_images_in_folder('Dataset/train/train/pair_00')



Classifying 0000_gt.jpg...
Image 0000_gt.jpg is classified as: No Moiré
Classifying 0000_moire.jpg...
Image 0000_moire.jpg is classified as: Moiré
Classifying 0001_gt.jpg...
Image 0001_gt.jpg is classified as: Moiré
Classifying 0001_moire.jpg...
Image 0001_moire.jpg is classified as: Moiré
Classifying 0002_gt.jpg...
Image 0002_gt.jpg is classified as: No Moiré
Classifying 0002_moire.jpg...
Image 0002_moire.jpg is classified as: Moiré
Classifying 0004_gt.jpg...
Image 0004_gt.jpg is classified as: Moiré
Classifying 0004_moire.jpg...
Image 0004_moire.jpg is classified as: Moiré
Classifying 0005_gt.jpg...
Image 0005_gt.jpg is classified as: Moiré
Classifying 0005_moire.jpg...
Image 0005_moire.jpg is classified as: Moiré
Classifying 0006_gt.jpg...
Image 0006_gt.jpg is classified as: Moiré
Classifying 0006_moire.jpg...
Image 0006_moire.jpg is classified as: Moiré
Classifying 0007_gt.jpg...
Image 0007_gt.jpg is classified as: Moiré
Classifying 0007_moire.jpg...
Image 0007_moire.jpg is classif

KeyboardInterrupt: 